<a href="https://colab.research.google.com/github/amantayal44/Hindi-to-English-NMT/blob/main/phase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PHASE 2 (NMT Hindi to English)

in phase 1, i tested various seq2seq model with encoder as biLSTM,LSTM,GRU,biGRU and similar decoder. I found that biGRU encoder with GRU decoder gives best scores on test data. In phase 1, I use vocabulary size of 8192(or 2^13) in this phase I will train model on different vocab size - 8192, 12228 (1.5*2^13) and 16384 (2^14) and then i will train model using different initialization - xavier uniform, xavier normal. I will also increase size of max length sentence in preprocessing from 20 t0 25

## Setup and Installing Libraries

I directly link my colab to google drive to easily load test file and store trained model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!pip install Morfessor
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"
!pip install nltk -U
!python3 -m spacy download en
!pip install revtok

Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1271 (delta 50), reused 54 (delta 25), pack-reused 1178
Receiving objects: 100% (1271/1271), 9.56 MiB | 8.19 MiB/s, done.
Resolving deltas: 100% (654/654), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 19.71 MiB/s, done.
Resolving deltas: 100% (51/51), done.
     |████████████████████████████████| 1.5MB 6.2MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist

In [ ]:
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from tqdm import tqdm
import nltk
nltk.download('wordnet')
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score
from indicnlp.tokenize import indic_tokenize 
import csv 
import re
import warnings
# warnings.filterwarnings("ignore") #uncomment only if code is done

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## Preprocessing

In [ ]:
orignal_dataset = []
with open("gdrive/MyDrive/train.csv",encoding="utf-8") as f:
  csv_reader = csv.reader(f, delimiter=',')
  i = 0
  for r in csv_reader:
    if i == 0:
      i = 1
      continue
    orignal_dataset.append([r[1],r[2]])

In [ ]:
len(orignal_dataset)

102322

In [ ]:
#non hindi symbols
non_hindi_chr = ['♫', '#', '$', '%', '&', '£', '¥', '§', '©', 'Â', 'è', 'Ã', '€','[',']']

In [ ]:
# function to clean data
def clean_data(dataset,max_length=20):
  # remove dataset that has non ascii character in english part and keep sentences that has length less than max_length
  new_dataset = []
  i = 0
  for data in dataset:
    l_1 = len(indic_tokenize.trivial_tokenize(data[0]))
    l_2 = len(data[1].split(" "))
    check_chr = True  
    for nh in non_hindi_chr:
      if nh in data[0]:
        check_chr = False
        break

    if re.search(r'[^\x00-\x7F]+',data[1]) == None and max(l_1,l_2) <= max_length and check_chr:
      new_dataset.append(data)
    elif i<5:
      if i == 0: print("Some removed datasets")
      i += 1
      print("{}. \"{}\" , \"{}\"".format(i,data[0],data[1]))
  print("removed {} of {} datasets".format(len(dataset)-len(new_dataset),len(dataset)))
  return new_dataset

In [ ]:
def capitalize_str(s):
  capt = True
  out = []
  for c in s:
    a = c
    if a in [" "]:
      pass
    elif ord(a) >= 97 and ord(a) <= 122:
      if capt: a = a.capitalize()
      capt = False
    elif a in ['?','.','!']:
      capt = True
    else:
      capt = False
    out.append(a)
  return "".join(out)

In [ ]:
#to preprocess english sentence
def preprocess_eng(sentence):
  sentence = sentence.lower().strip() #lower case letters
  # removing shortforms
  sentence = re.sub(r"i'm","i am",sentence)
  sentence = re.sub(r"let's","let us",sentence)
  sentence = re.sub(r"\'ll", " will", sentence)
  sentence = re.sub(r"\'ve", " have", sentence)
  sentence = re.sub(r"\'re", " are", sentence)
  sentence = re.sub(r"\'d", " would", sentence)
  sentence = re.sub(r"\'re", " are", sentence)
  sentence = re.sub(r"n't"," not",sentence)

  sentence = re.sub(r"([?.!,])", r" \1 ", sentence) #creating space b/w punctuation
  sentence = re.sub(r'[" "]+', " ", sentence) # removing multiple places
  sentence = sentence.strip()
  return sentence

# some corresponding postprocess to increase score
def postprocess_eng(sentence,remove_unk=False):
  # sentence = capitalize_str(sentence) #capitalize first
  sentence = sentence.capitalize()
  sentence = re.sub(r" i ",r" I ",sentence) #changes small 'i' to capital 'I' in sentence
  sentence = re.sub(r" ([?.!,])",r"\1",sentence) #remove space b/w last word and punctuation
  if remove_unk: sentence = re.sub(r" <unk> ",r" ",sentence) # to remove <unk> token
  return sentence


In [ ]:
#preprocess data using above functions and check bleu score of preprocessed output
def data_preprocessing(dataset,max_length=20):
  new_dataset = []
  for data in dataset:
    new_dataset.append([data[0],preprocess_eng(data[1]),data[1]])
  new_dataset = clean_data(new_dataset,max_length)
  # comparing change in bleu score and meteor score
  total_bleu_score = 0
  total_meteor_score = 0
  for i in tqdm(range(len(new_dataset))):
    total_bleu_score += sentence_bleu([new_dataset[i][2].split(" ")], postprocess_eng(new_dataset[i][1]).split(" "))
    total_meteor_score += single_meteor_score(new_dataset[i][2],postprocess_eng(new_dataset[i][1]))

  l = len(new_dataset)
  print("\nbleu score {}".format(round(total_bleu_score/l,2)))
  print("meteor score {}".format(round(total_meteor_score/l,2)))

  return new_dataset

In [ ]:
dataset = data_preprocessing(orignal_dataset,max_length=25)

Some removed datasets
1. "एल सालवाडोर मे, जिन दोनो पक्षों ने सिविल-युद्ध से वापसी ली, उन्होंने वही काम किये जो कैदियों की कश्मकश के निदान हैं।" , "in el salvador , both sides that withdrew from their civil war took moves that had been proven to mirror a prisoner's dilemma strategy ."
2. "पर मेरे लिए उसका यहुदी विरोधी होना उसके कार्यों को और भी प्रशंसनीय बनाता है क्योंकि उसके पास भी पक्षपात करने के वही कारण थे जो बाकी फौजियों के पास थे पर उसकी सच जानने और उसे बनाए रखने की प्रेरणा सबसे ऊपर थी" , "but personally , for me , the fact that picquart was anti-semitic actually makes his actions more admirable , because he had the same prejudices , the same reasons to be biased as his fellow officers , but his motivation to find the truth and uphold it trumped all of that ."
3. "तो स्मार्ट में, हमारे पास लक्ष्य के अलावा, मलेरिया टीका विकसित करने के, हम अफ्रीकी वैज्ञानिकों को भी प्रशिक्षण दे रहे हैं, क्योंकि अफ्रीका में बीमारी का बोझ काफी ज़्यादा है, और आपको उन लोगों की आवश्यकता है जो सीमाओं को आ

  0%|          | 0/89076 [00:00<?, ?it/s]

removed 13246 of 102322 datasets


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  0%|          | 7/89076 [00:01<5:22:34,  4.60it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram or


bleu score 0.58
meteor score 0.92


In [ ]:
from sklearn.model_selection import train_test_split
train_set,val_set = train_test_split(dataset,test_size=0.2,random_state=42)
val_set,test_set = train_test_split(val_set,test_size=0.5,random_state=42)
len(train_set),len(val_set),len(test_set)

(71260, 8908, 8908)

## Tokenization

spacy for english and indic_tokenize for hindi

In [ ]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
import torch
import random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
eng_tokenizer=get_tokenizer('spacy', language='en')
hindi_tokenizer = get_tokenizer(indic_tokenize.trivial_tokenize)

In [ ]:
def get_vocab(dataset,eng_tokenizer,hindi_tokenizer,max_size_eng=5000,max_size_hindi=5000):
  eng_counter = Counter()
  hindi_counter = Counter()
  for data in tqdm(dataset):
    eng_counter.update(eng_tokenizer(data[1]))
    hindi_counter.update(hindi_tokenizer(data[0]))
    
  eng_vocab = Vocab(eng_counter,max_size=max_size_eng,specials=('<pad>','<unk>','<eos>','<sos>'))
  hindi_vocab = Vocab(hindi_counter,max_size=max_size_hindi,specials=('<pad>','<unk>','<eos>','<sos>'))
  return eng_vocab,hindi_vocab,eng_counter,hindi_counter

In [ ]:
# 3 vocab (2**13,1.5*2**13,2**14)
eng_vocab_1,hindi_vocab_1,eng_counter,hindi_counter = get_vocab(train_set,eng_tokenizer,hindi_tokenizer,2**13,2**13)
eng_vocab_2,hindi_vocab_2,_,_ = get_vocab(train_set,eng_tokenizer,hindi_tokenizer,1.5*(2**13),1.5*(2**13))
eng_vocab_3,hindi_vocab_3,_,_ = get_vocab(train_set,eng_tokenizer,hindi_tokenizer,2**14,2**14)

100%|██████████| 71260/71260 [00:04<00:00, 16003.83it/s]


In [ ]:
len(eng_counter),len(hindi_counter)

(22933, 31245)

In [ ]:
print(len(eng_vocab_1),len(hindi_vocab_1))
print(len(eng_vocab_2),len(hindi_vocab_2))
print(len(eng_vocab_3),len(hindi_vocab_3))

8196 8196
12292 12292
16388 16388


In [ ]:
def tokenize(dataset,eng_tokenizer,hindi_tokenizer,eng_vocab,hindi_vocab):
  tokenized_data = []
  for data in dataset:
    eng_data = torch.tensor([eng_vocab['<sos>']]+[eng_vocab[t] for t in eng_tokenizer(data[1])]+[eng_vocab['<eos>']], dtype=torch.long)
    hindi_data = torch.tensor([hindi_vocab['<sos>']]+[hindi_vocab[t] for t in hindi_tokenizer(data[0])]+[hindi_vocab['<eos>']], dtype=torch.long)
    tokenized_data.append([hindi_data,eng_data])
  return tokenized_data

In [ ]:
#1
tokenized_train_data_1= tokenize(train_set,eng_tokenizer,hindi_tokenizer,eng_vocab_1,hindi_vocab_1)
tokenized_val_data_1= tokenize(val_set,eng_tokenizer,hindi_tokenizer,eng_vocab_1,hindi_vocab_1)
tokenized_test_data_1 = tokenize(test_set,eng_tokenizer,hindi_tokenizer,eng_vocab_1,hindi_vocab_1)
#2
tokenized_train_data_2= tokenize(train_set,eng_tokenizer,hindi_tokenizer,eng_vocab_2,hindi_vocab_2)
tokenized_val_data_2= tokenize(val_set,eng_tokenizer,hindi_tokenizer,eng_vocab_2,hindi_vocab_2)
tokenized_test_data_2 = tokenize(test_set,eng_tokenizer,hindi_tokenizer,eng_vocab_2,hindi_vocab_2)
#3
tokenized_train_data_3= tokenize(train_set,eng_tokenizer,hindi_tokenizer,eng_vocab_3,hindi_vocab_3)
tokenized_val_data_3= tokenize(val_set,eng_tokenizer,hindi_tokenizer,eng_vocab_3,hindi_vocab_3)
tokenized_test_data_3 = tokenize(test_set,eng_tokenizer,hindi_tokenizer,eng_vocab_3,hindi_vocab_3)

creating batches of data and padding them using pytorch dataloader

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

BATCH_SIZE = 128
pad_hindi = hindi_vocab_1['<pad>']
pad_eng = eng_vocab_1['<pad>']

In [ ]:
print(hindi_vocab_1['<pad>'],hindi_vocab_2['<pad>'],hindi_vocab_3['<pad>'])
print(hindi_vocab_1['<eos>'],hindi_vocab_2['<eos>'],hindi_vocab_3['<eos>'])
print(hindi_vocab_1['<sos>'],hindi_vocab_2['<sos>'],hindi_vocab_3['<sos>'])
print(hindi_vocab_1['<unk>'],hindi_vocab_2['<unk>'],hindi_vocab_3['<unk>'])

# all has same values for special tokens

0 0 0
2 2 2
3 3 3
1 1 1


In [ ]:
def get_data(data):
  hindi_data = []
  eng_data = []
  for hindi_sen,eng_sen in data:
    hindi_data.append(hindi_sen)
    eng_data.append(eng_sen)
  hindi_data = pad_sequence(hindi_data,padding_value=pad_hindi)
  eng_data = pad_sequence(eng_data,padding_value=pad_eng)
  return hindi_data,eng_data

In [ ]:
#1
train_data_1 = DataLoader(tokenized_train_data_1, batch_size=BATCH_SIZE,shuffle=True,collate_fn=get_data)
val_data_1 = DataLoader(tokenized_val_data_1, batch_size=BATCH_SIZE,shuffle=True,collate_fn=get_data)
test_data_1 = DataLoader(tokenized_test_data_1, batch_size=BATCH_SIZE,shuffle=True,collate_fn=get_data)
#2
train_data_2 = DataLoader(tokenized_train_data_2, batch_size=BATCH_SIZE,shuffle=True,collate_fn=get_data)
val_data_2 = DataLoader(tokenized_val_data_2, batch_size=BATCH_SIZE,shuffle=True,collate_fn=get_data)
test_data_2 = DataLoader(tokenized_test_data_2, batch_size=BATCH_SIZE,shuffle=True,collate_fn=get_data)
#3
train_data_3 = DataLoader(tokenized_train_data_3, batch_size=BATCH_SIZE,shuffle=True,collate_fn=get_data)
val_data_3 = DataLoader(tokenized_val_data_3, batch_size=BATCH_SIZE,shuffle=True,collate_fn=get_data)
test_data_3 = DataLoader(tokenized_test_data_3, batch_size=BATCH_SIZE,shuffle=True,collate_fn=get_data)

##Model

seq2seq with bigru encoder and gru decoder 

In [ ]:
from torch import nn
from torch.nn import LSTM,GRU,Linear,Embedding
import torch.optim as optim
import time

In [ ]:
e_vocab_size_1 = len(hindi_vocab_1)
d_vocab_size_1 = len(eng_vocab_1)
e_vocab_size_2 = len(hindi_vocab_2)
d_vocab_size_2 = len(eng_vocab_2)
e_vocab_size_3 = len(hindi_vocab_3)
d_vocab_size_3 = len(eng_vocab_3)


In [ ]:
class biEncoder(nn.Module):
  def __init__(self,vocab_size,emb_size=512,hid_size=512,dropout=0.5,out=512):
    super().__init__()
    self.vocab_size = vocab_size
    self.hid_size = hid_size
    self.embedding = Embedding(vocab_size,emb_size)
    self.dropout = nn.Dropout(dropout)
    self.rnn = GRU(emb_size,hid_size,bidirectional=True) #bidirectional
    self.out = Linear(2*hid_size,out)
  
  def forward(self,input):
    embedded = self.embedding(input)
    embedded = self.dropout(embedded)
    outputs,h = self.rnn(embedded)
    h = self.out(torch.cat((h[0], h[1]), dim = 1)).unsqueeze(0) #concat hidden values of 2 direction and forward layer for output dim
    return h

class biDecoder(nn.Module):
  def __init__(self,vocab_size,emb_size=256,hid_size=512,dropout=0.5):
    super().__init__()
    self.vocab_size = vocab_size
    self.hid_size = hid_size
    self.embedding = Embedding(vocab_size,emb_size)
    self.dropout = nn.Dropout(dropout)
    self.rnn = GRU(emb_size,hid_size) #unidirectional
    self.out = Linear(hid_size,vocab_size)

  def forward(self,input,h):
    input = input.view(1,-1)
    embedded = self.embedding(input)
    embedded = self.dropout(embedded)
    output,h = self.rnn(embedded,h)
    output = self.out(output.squeeze(0))
    return output,h

class biseq2seq(nn.Module):
  def __init__(self,device,e_vocab_size,d_vocab_size,emb_size=256,hid_size_e=512,hid_size_d=512,dropout=0.5):
    super().__init__()
    self.d_vocab_size = d_vocab_size
    self.e_vocab_size = e_vocab_size
    self.encoder = biEncoder(e_vocab_size,emb_size,hid_size_e,dropout,out=hid_size_d)
    self.decoder = biDecoder(d_vocab_size,emb_size,hid_size_d,dropout)
    self.device = device

  def forward(self,input,target,forcing = 0.5):
    batch_size = target.shape[1]
    len = target.shape[0]
    #output of dim [len,batch_size,vocab_size]
    output = torch.zeros(len,batch_size,self.d_vocab_size).to(self.device)
    h = self.encoder(input) #encoder output

    input = target[0] #taking first token of target as input (which is <sos>)
    for i in range(len-1): 
      out,h = self.decoder(input,h) #encoder output
      output[i+1] = out
      force = random.random() < forcing #either choose next target as input or predicted value by model
      #using teacher force helps model to generate better results when only first word is ssen
      # improves generating power of rnn model
      if force: input = target[i+1]
      else: input = out.argmax(1) 
    
    return output

In [ ]:
#using norm clipping to avoid exploding gradient in GRU
#using batch gradient descent
def train(model,dataset,optimizer,loss_fn,clip=1):
  model.train()
  total_loss = 0
  for input,target in dataset:
    # initializing optimizer
    optimizer.zero_grad()
    input = input.to(device)
    target = target.to(device)
    # ouput from model
    output = model(input,target)
    # from dim [len,batch_size] to [len*batch_size]
    target = target[1:].view(-1) #ignoring 0th values as its of <sos> and 0 in output
    #from from dim [len,batch_size,vocab_size] to [len*batch_size,vocab_size]
    output = output[1:].view(-1,output.shape[-1])
    # calculating loss
    batch_loss = loss_fn(output,target)
    # back propogation 
    batch_loss.backward()
    # norm clipping to avoid exploding gradients
    if clip is not None:
      torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    #upgrading gradients
    optimizer.step()
    total_loss += batch_loss.item()
  
  return total_loss/len(dataset)

def evaluate(model,dataset,loss_fn):
  model.eval() #to avoid dropout and other things use during training
  total_loss = 0
  with torch.no_grad(): #not compute graidents
    for input,target in dataset:
      input = input.to(device)
      target = target.to(device)
      output = model(input,target,0)
      target = target[1:].view(-1)
      output = output[1:].view(-1,output.shape[-1])
      # calculating loss on eval set
      batch_loss = loss_fn(output,target)
      total_loss += batch_loss.item()
  
  return total_loss/len(dataset)

In [ ]:
#if validation loss increase by more than stop it terminates training
def fit(model,train_data,val_data,optimizer,loss_fn,name="model",EPOCHS=10,clip=1,stop=0.1,test_data=None):
    history = [] #used for futher analysis and graph
    min_val = 10000 #used in finding parameters with least val set loss
    for epoch in range(EPOCHS):
        start = time.time()
        train_loss = train(model, train_data, optimizer,loss_fn,clip)
        val_loss = evaluate(model, val_data,loss_fn)  
        end = time.time()
        print("train loss: {:.3f} val loss: {:.3f}".format(train_loss,val_loss))
        t = end - start
        print("time taken by {} epoch {} min {} s".format(epoch+1,int(t/60),int(t%60)))
        history.append({
            "epoch":epoch,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "time": t,
        })
        if val_loss<min_val:
            min_val = val_loss
             #saving best model
            torch.save(model.state_dict(), name+'_best.pt')
        # stop training if val_loss increase more than by stop
        if val_loss > min_val*(1+stop):
            break
    #saving model on last epoch
    torch.save(model.state_dict(), name+'.pt')
    #evaluating both best and last model on test set
    if test_data is not None:
        test_loss = evaluate(model,test_data,loss_fn)
        model.load_state_dict(torch.load(name+'_best.pt'))
        test_loss_best = evaluate(model,test_data,loss_fn)
        print("test loss: {:.3f} test loss on best val: {:.3f}".format(test_loss,test_loss_best))
    
    return history

In [ ]:
#using embedding size of 512 and hidden size of 512 also
model_1 = biseq2seq(device,e_vocab_size_1,d_vocab_size_1,emb_size=512,hid_size_e=512,hid_size_d=512).to(device)
model_2 = biseq2seq(device,e_vocab_size_2,d_vocab_size_2,emb_size=512,hid_size_e=512,hid_size_d=512).to(device)
model_3 = biseq2seq(device,e_vocab_size_3,d_vocab_size_3,emb_size=512,hid_size_e=512,hid_size_d=512).to(device)

In [ ]:
#counting no. of parameters in model
def parameters_count(model):
    return sum(parameter.numel() for parameter in model.parameters() if parameter.requires_grad)

In [ ]:
print(parameters_count(model_1)) #17.8 M
print(parameters_count(model_2)) #24.1 M
print(parameters_count(model_3)) #30.4 M

17849860
24145412
30440964


In [ ]:
#using cross entropy loss with ignoring output fot padded values
loss_fn = nn.CrossEntropyLoss(ignore_index = pad_eng)
#using adam optimizer (as dicussed in class)
optimizer_1 = optim.Adam(model_1.parameters())
optimizer_2 = optim.Adam(model_2.parameters())
optimizer_3 = optim.Adam(model_3.parameters())

In [ ]:
#first using simple initialization from gaussian distribution with mean 0 and std 0.01
def initialize_normal(model):
  for layer,parameter in model.named_parameters():
    #initializing variable
    if "weight" in layer:
      nn.init.normal_(parameter.data, mean=0, std=0.01)
    #initializing constant
    else:
      nn.init.constant_(parameter.data, 0)

In [ ]:
#initializing models
model_1.apply(initialize_normal)
model_2.apply(initialize_normal)
model_3.apply(initialize_normal)

biseq2seq(
  (encoder): biEncoder(
    (embedding): Embedding(16388, 512)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(512, 512, bidirectional=True)
    (out): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): biDecoder(
    (embedding): Embedding(16388, 512)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(512, 512)
    (out): Linear(in_features=512, out_features=16388, bias=True)
  )
)

##Training

In [ ]:
#Traing 1st model (8k vocab)
history_1 = fit(model_1,train_data_1,val_data_1,optimizer_1,loss_fn,name="model_1",test_data=test_data_1)

train loss: 5.113 val loss: 4.880
time taken by 1 epoch -2 min 56 s
train loss: 4.318 val loss: 4.523
time taken by 2 epoch -2 min 56 s
train loss: 3.896 val loss: 4.322
time taken by 3 epoch -2 min 57 s
train loss: 3.560 val loss: 4.202
time taken by 4 epoch -2 min 56 s
train loss: 3.268 val loss: 4.124
time taken by 5 epoch -2 min 56 s
train loss: 3.024 val loss: 4.096
time taken by 6 epoch -2 min 56 s
train loss: 2.819 val loss: 4.104
time taken by 7 epoch -2 min 56 s
train loss: 2.637 val loss: 4.150
time taken by 8 epoch -2 min 56 s
train loss: 2.498 val loss: 4.166
time taken by 9 epoch -2 min 56 s
train loss: 2.368 val loss: 4.219
time taken by 10 epoch -2 min 56 s
test loss: 4.225 test loss on best val: 4.106


In [ ]:
#donot run (not best)
#Traing 2nd model (12k vocab)
history_2 = fit(model_2,train_data_2,val_data_2,optimizer_2,loss_fn,name="model_2",test_data=test_data_2)

train loss: 5.253 val loss: 4.982
time taken by 1 epoch -2 min 11 s
train loss: 4.435 val loss: 4.630
time taken by 2 epoch -2 min 11 s
train loss: 4.013 val loss: 4.424
time taken by 3 epoch -2 min 11 s
train loss: 3.656 val loss: 4.319
time taken by 4 epoch -2 min 11 s
train loss: 3.341 val loss: 4.234
time taken by 5 epoch -2 min 11 s
train loss: 3.058 val loss: 4.237
time taken by 6 epoch -2 min 11 s
train loss: 2.824 val loss: 4.267
time taken by 7 epoch -2 min 11 s
train loss: 2.634 val loss: 4.281
time taken by 8 epoch -2 min 10 s
train loss: 2.471 val loss: 4.342
time taken by 9 epoch -2 min 11 s
test loss: 4.362 test loss on best val: 4.239


In [ ]:
#donot run (not best)
#Traing 3rd model (16k vocab)
history_3 = fit(model_3,train_data_3,val_data_3,optimizer_3,loss_fn,name="model_3",test_data=test_data_3)

train loss: 5.161 val loss: 4.959
time taken by 1 epoch 3 min 34 s
train loss: 4.414 val loss: 4.654
time taken by 2 epoch 3 min 34 s
train loss: 3.971 val loss: 4.431
time taken by 3 epoch 3 min 33 s
train loss: 3.581 val loss: 4.328
time taken by 4 epoch 3 min 33 s
train loss: 3.243 val loss: 4.250
time taken by 5 epoch 3 min 33 s
train loss: 2.947 val loss: 4.287
time taken by 6 epoch 3 min 33 s
train loss: 2.715 val loss: 4.353
time taken by 7 epoch 3 min 33 s
test loss: 4.385 test loss on best val: 4.278


In [ ]:
#output by model for hindi sentence
def inference(model,sentence,eng_vocab,hindi_vocab,max_len=40):
  model.eval() #in eval model
  #dim from [len] to [len,1] (as not in batches)
  sentence = sentence.unsqueeze(1).to(device)
  output = [eng_vocab['<sos>']] #output vector
  with torch.no_grad():
    h = model.encoder(sentence) #encoder output
    for i in range(max_len):
      input = torch.tensor([output[-1]],dtype=torch.long).to(device) #input is last output
      out,h = model.decoder(input,h) #decoder output
      prediction = out.argmax(1).item() #word with most value in final layer
      if prediction == eng_vocab['<eos>']: #break if <eos> token is found (end of sentence)
        break
      output.append(prediction)
  return output[1:] #returning output (ignoring <sos> token)

In [ ]:
#calculates bleu and meteor score on test data 
def score_data(test,model,inference,eng_vocab,hindi_vocab,hindi_tokenizer,remove_unk=False):
    total_bleu_score_p = 0
    total_meteor_score_p = 0
    total_bleu_score = 0
    total_meteor_score = 0
    for i in tqdm(range(len(test))):
        data = test[i]
        tokenized = torch.tensor([hindi_vocab['<sos>']]+[hindi_vocab[t] for t in hindi_tokenizer(data[0])]+[hindi_vocab['<eos>']], dtype=torch.long)
        output = inference(model,tokenized,eng_vocab,hindi_vocab)
        output = " ".join([eng_vocab.itos[t] for t in output])
        #without postprocessing on output after preprocessing
        total_bleu_score += sentence_bleu([data[1].split(" ")], output.split(" "))
        total_meteor_score += single_meteor_score(data[1],output)
        #with postprocessing on output from actual data
        total_bleu_score_p += sentence_bleu([data[2].split(" ")], postprocess_eng(output,remove_unk).split(" "))
        total_meteor_score_p += single_meteor_score(data[2],postprocess_eng(output,remove_unk))

    l = len(test)
    print("\nbleu score {:.4f}, bleu score with on actual {:.4f}".format(total_bleu_score/l,total_bleu_score_p/l))
    print("meteor score {:.4f}, meteor score with on actual {:.4f}".format(total_meteor_score/l,total_meteor_score_p/l))

In [ ]:
#scores on model_1
score_data(test_set,model_1,inference,eng_vocab_1,hindi_vocab_1,hindi_tokenizer)

  0%|          | 0/8908 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use 


bleu score 0.0328, bleu score with on actual 0.0074
meteor score 0.2592, meteor score with on actual 0.1789


In [ ]:
#donot run (not best)
#scores on model_2
score_data(test_set,model_2,inference,eng_vocab_2,hindi_vocab_2,hindi_tokenizer)

  0%|          | 0/8908 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use 


bleu score 0.0299, bleu score with on actual 0.0057
meteor score 0.2506, meteor score with on actual 0.1686


In [ ]:
#donot run (not best)
#scores on model_3
score_data(test_set,model_3,inference,eng_vocab_3,hindi_vocab_3,hindi_tokenizer)

  0%|          | 0/8908 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use 


bleu score 0.0280, bleu score with on actual 0.0053
meteor score 0.2601, meteor score with on actual 0.1725


model with 8k embedding size perform, hence too much overfitting hurts performance of model

Now we will use different initialization like xavier_normal,xavier_unifrom and simple uniform on model

In [ ]:
def initialize_xavier_uniform(model):
  for layer,parameter in model.named_parameters():
    #initializing variable
    if "weight" in layer:
      nn.init.xavier_uniform_(parameter.data)
    #initializing constant with 0
    else:
      nn.init.constant_(parameter.data, 0)

def initialize_xavier_normal(model):
  for layer,parameter in model.named_parameters():
    #initializing variable
    if "weight" in layer:
      nn.init.xavier_normal_(parameter.data)
     #initializing constant with 0
    else:
      nn.init.constant_(parameter.data, 0)

def initialize_uniform(model):
  for layer,parameter in model.named_parameters():
    #initializing variable
    if "weight" in layer:
      nn.init.uniform_(parameter.data, -0.08, 0.08)
     #initializing constant with 0
    else:
      nn.init.constant_(parameter.data, 0)

In [ ]:
optimizer_1 = optim.Adam(model_1.parameters())
model_1.apply(initialize_xavier_normal)

biseq2seq(
  (encoder): biEncoder(
    (embedding): Embedding(8196, 512)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(512, 512, bidirectional=True)
    (out): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): biDecoder(
    (embedding): Embedding(8196, 512)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(512, 512)
    (out): Linear(in_features=512, out_features=8196, bias=True)
  )
)

In [ ]:
#donot run (not best)
history_1_1 = fit(model_1,train_data_1,val_data_1,optimizer_1,loss_fn,name="model_1_init_1",test_data=test_data_1)

train loss: 4.909 val loss: 4.703
time taken by 1 epoch 2 min 3 s
train loss: 4.157 val loss: 4.432
time taken by 2 epoch 2 min 3 s
train loss: 3.773 val loss: 4.254
time taken by 3 epoch 2 min 3 s
train loss: 3.433 val loss: 4.121
time taken by 4 epoch 2 min 3 s
train loss: 3.166 val loss: 4.074
time taken by 5 epoch 2 min 3 s
train loss: 2.933 val loss: 4.063
time taken by 6 epoch 2 min 3 s
train loss: 2.735 val loss: 4.124
time taken by 7 epoch 2 min 3 s
train loss: 2.583 val loss: 4.118
time taken by 8 epoch 2 min 3 s
train loss: 2.443 val loss: 4.177
time taken by 9 epoch 2 min 3 s
test loss: 4.173 test loss on best val: 4.066


In [ ]:
#donot run (not best)
score_data(test_set,model_1,inference,eng_vocab_1,hindi_vocab_1,hindi_tokenizer)

  0%|          | 0/8908 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use 


bleu score 0.0370, bleu score with on actual 0.0076
meteor score 0.2706, meteor score with on actual 0.1891


In [ ]:
model_1.apply(initialize_xavier_uniform)
optimizer_1 = optim.Adam(model_1.parameters())

In [ ]:
#donot run (not best)
history_1_2 = fit(model_1,train_data_1,val_data_1,optimizer_1,loss_fn,name="model_1_init_2",test_data=test_data_1)

train loss: 4.895 val loss: 4.667
time taken by 1 epoch 2 min 3 s
train loss: 4.109 val loss: 4.354
time taken by 2 epoch 2 min 3 s
train loss: 3.696 val loss: 4.180
time taken by 3 epoch 2 min 3 s
train loss: 3.370 val loss: 4.093
time taken by 4 epoch 2 min 3 s
train loss: 3.102 val loss: 4.040
time taken by 5 epoch 2 min 3 s
train loss: 2.875 val loss: 4.052
time taken by 6 epoch 2 min 3 s
train loss: 2.671 val loss: 4.098
time taken by 7 epoch 2 min 3 s
train loss: 2.513 val loss: 4.103
time taken by 8 epoch 2 min 3 s
train loss: 2.368 val loss: 4.187
time taken by 9 epoch 2 min 3 s
test loss: 4.202 test loss on best val: 4.052


In [ ]:
#donot run (not best)
score_data(test_set,model_1,inference,eng_vocab_1,hindi_vocab_1,hindi_tokenizer)

  0%|          | 0/8908 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use 


bleu score 0.0351, bleu score with on actual 0.0076
meteor score 0.2646, meteor score with on actual 0.1834


In [ ]:
model_1.apply(initialize_uniform)
optimizer_1 = optim.Adam(model_1.parameters())

In [ ]:
history_1_2 = fit(model_1,train_data_1,val_data_1,optimizer_1,loss_fn,name="model_1_init_2",test_data=test_data_1)

train loss: 4.648 val loss: 4.495
time taken by 1 epoch 2 min 3 s
train loss: 3.895 val loss: 4.177
time taken by 2 epoch 2 min 4 s
train loss: 3.477 val loss: 4.042
time taken by 3 epoch 2 min 3 s
train loss: 3.150 val loss: 3.999
time taken by 4 epoch 2 min 3 s
train loss: 2.899 val loss: 3.939
time taken by 5 epoch 2 min 3 s
train loss: 2.673 val loss: 3.972
time taken by 6 epoch 2 min 3 s
train loss: 2.491 val loss: 4.056
time taken by 7 epoch 2 min 3 s
test loss: 4.055 test loss on best val: 3.949


In [ ]:
score_data(test_set,model_1,inference,eng_vocab_1,hindi_vocab_1,hindi_tokenizer)

  0%|          | 0/8908 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use 


bleu score 0.0412, bleu score with on actual 0.0085
meteor score 0.2810, meteor score with on actual 0.1943


model with uniform initialization gives best scores and same initialization is used in original seq2seq paper

In [ ]:
#saving model on drive (ignore)
!cp model_1_init_2_best.pt gdrive/MyDrive/cs779_model/

In [ ]:
sample = []
with open("hindistatements-2.csv",encoding="utf-8") as f:
  csv_reader = csv.reader(f, delimiter=',')
  i = 0
  for r in csv_reader:
    if i == 0:
      i = 1
      continue
    sample.append(r[2])

In [ ]:
def final_result(model,inference,sample,hindi_tokenizer,hindi_vocab,eng_vocab):
  result = []
  for s in sample:
    hindi_s = torch.tensor([hindi_vocab['<sos>']]+[hindi_vocab[t] for t in hindi_tokenizer(s)]+[hindi_vocab['<eos>']], dtype=torch.long)
    output = inference(model,hindi_s,eng_vocab,hindi_vocab)
    output = " ".join([eng_vocab.itos[t] for t in output])
    output = postprocess_eng(output)
    result.append(output)
  return result

In [ ]:
result = final_result(model_1,inference,sample,hindi_tokenizer,hindi_vocab_1,eng_vocab_1)

In [ ]:
sample[21], result[21]

('मैं पहले एक इंजीनियर था.', 'I was a first one')

In [ ]:
f = open("answer.txt", "w")
for s in result:
  f.write(s+"\n")
f.close()